In [16]:
####Heterogeneous Infomation Network Embbedding with Variable order Tucker Decomposition####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import itertools
import scipy
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from datetime import time, datetime, timedelta
from scipy import sparse
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
import re
import MeCab
import neologdn
import sys

np.random.seed(98537)

In [78]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no, pattern):
    z_id = np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1)
    return z_id
    if pattern==1:
        Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
        return z_id, Z

In [17]:
####データの設定####
##シナリオの読み込みとクレンジング
#データの読み込み
company = pd.read_csv("D:/Statistics/data/scenario/company.csv")
scenario_df1 = pd.read_csv("D:/Statistics/data/scenario/level1_scenario.csv")
scenario_df2 = pd.read_csv("D:/Statistics/data/scenario/level2_scenario.csv")
factor_df = pd.read_csv("D:/Statistics/data/scenario/Factor.csv")

#読み込んだデータをクレンジング
scenario_df1.columns = (scenario_df1.columns).str.replace(" ", "")
scenario_df2.columns = (scenario_df2.columns).str.replace(" ", "")
factor_df.columns = (factor_df.columns).str.replace(" ", "")
company.columns = (company.columns).str.replace(" ", "")
columns1 = np.array(scenario_df1.columns)
columns2 = np.array(scenario_df2.columns)
columns3 = np.array(factor_df.columns)
columns4 = np.array(company.columns)
for j in range(columns1.shape[0]):
    scenario_df1[columns1[j]] = scenario_df1[columns1[j]].str.replace("\"| ", "")
for j in range(columns2.shape[0]):
    scenario_df2[columns2[j]] = scenario_df2[columns2[j]].str.replace("\"| ", "")
for j in range(columns3.shape[0]):
    factor_df[columns3[j]] = factor_df[columns3[j]].str.replace("\"| ", "")
scenario_df1["c.ticker"] = np.array(scenario_df1["c.ticker"], dtype="int")
scenario_df2["c.ticker"] = np.array(scenario_df2["c.ticker"], dtype="int")
scenario_df1["c.name"] = pd.merge(scenario_df1[["c.ticker"]], company, on="c.ticker", how="left")["c.name"]
scenario_df2["c.name"] = pd.merge(scenario_df2[["c.ticker"]], company, on="c.ticker", how="left")["c.name"]
scenario_df21 = scenario_df2[["f2.area", "f2.predicate", "f2.item", "f2.trend", "f2.element", "a.name", "a.vector", "c.name", "c.ticker"]]
scenario_df21 = scenario_df21.rename(columns={"f2.area": "f.area", "f2.predicate": "f.predicate", 
                                              "f2.item": "f.item", "f2.trend": "f.trend", "f2.element": "f.element"})
scenario_df1 = pd.concat((scenario_df1, scenario_df21), axis=0)
del scenario_df21

In [18]:
##解析対象のレコードを抽出して、名寄を行う
#scenario1のレコードを抽出
index_dup = np.array(np.where(scenario_df1[["f.item", "f.element", "c.name", "c.ticker"]].duplicated()==False)[0], dtype="int")
scenario1 = scenario_df1[["f.item", "f.element", "c.name", "c.ticker"]].iloc[index_dup]
scenario1.index = np.arange(scenario1.shape[0])

#scenario2のレコードを抽出
index_dup = np.array(np.where(scenario_df2[["f1.item", "f1.element", "f2.item", 
                                            "f2.element", "c.name", "c.ticker"]].duplicated()==False)[0], dtype="int")
scenario2 = scenario_df2[["f1.item", "f1.element", "f2.item", "f2.element", "c.name", "c.ticker"]].iloc[index_dup]
scenario2.index = np.arange(scenario2.shape[0])

#factorのレコードを抽出
index_dup = np.array(np.where(factor_df[["f1.item", "f1.element", "f2.item", "f2.element"]].duplicated()==False)[0], dtype="int")
factor = factor_df[["f1.item", "f1.element", "f2.item", "f2.element"]].iloc[index_dup]
factor.index = np.arange(factor.shape[0])

In [54]:
##要因および企業のidを生成
#itemのidを作成
temp = pd.concat((scenario1["f.item"], scenario2["f1.item"], 
                  scenario2["f2.item"], factor["f1.item"], factor["f2.item"]), axis=0)
temp_freq = temp.value_counts()
index = np.where(temp_freq >= 3)[0]
target = np.array(temp_freq.index[index]); item_freq = temp_freq.iloc[index]
temp = temp.iloc[np.where(np.in1d(np.array(temp), target))[0]]
item_entity = np.append(np.unique(temp)[1:], "")
item_id = np.arange(item_entity.shape[0])
item = item_entity.shape[0]-1
item_df = pd.DataFrame({"id": item_id, "item": item_entity})

#elementのidを作成
temp = pd.concat((scenario1["f.element"], scenario2["f1.element"], 
                  scenario2["f2.element"], factor["f1.element"], factor["f2.element"]), axis=0)
temp_freq = temp.value_counts()
index = np.where(temp_freq >= 3)[0]
target = np.array(temp_freq.index[index]); element_freq = temp_freq.iloc[index]
temp = temp.iloc[np.where(np.in1d(np.array(temp), target))[0]]
element_entity = np.append(np.unique(temp)[1:], "")
element_id = np.arange(element_entity.shape[0])
element = element_entity.shape[0]-1
element_df = pd.DataFrame({"id": element_id, "element": element_entity})

#企業のidを作成
temp = pd.concat((scenario1[["c.name", "c.ticker"]], scenario2[["c.name", "c.ticker"]]), axis=0)
temp_freq = temp["c.ticker"].value_counts()
index = np.where(temp_freq >= 5)[0]
target = np.array(temp_freq.index[index]); company_freq = temp_freq.iloc[index]
temp = temp.iloc[np.where(np.in1d(np.array(temp["c.ticker"]), target))[0]]
temp.index = np.arange(temp.shape[0])
company_df = temp.iloc[np.where(temp.duplicated()==False)[0]]
company_df = company_df.sort_values("c.ticker")
company_df.index = np.arange(company_df.shape[0])
company_df["id"] = np.arange(company_df.shape[0])
company_df = company_df[["id", "c.ticker", "c.name"]]
company_df.columns = ["id", "ticker", "company"]
company_id = np.array(company_df["id"], dtype="int")

#頻度のデータフレームを作成
company_freq = pd.DataFrame({"ticker": np.array(company_freq.index, dtype="int"), "freq": np.array(company_freq, dtype="int")})
company_freq = pd.merge(company_freq0, company_df, on="ticker", how="left")[["id", "ticker", "company", "freq"]]
item_freq = pd.DataFrame({"item": np.array(item_freq.index), "freq": np.array(item_freq, dtype="int")})
item_freq = pd.merge(item_freq0, item_df, on="item", how="left")[["id", "item", "freq"]]
element_freq = pd.DataFrame({"element": np.array(element_freq.index), "freq": np.array(element_freq, dtype="int")})
element_freq = pd.merge(element_freq, element_df, on="element", how="left")[["id", "element", "freq"]]

In [56]:
##要因-企業間のネットワークデータを生成
#要因-企業間のネットワークを生成
item_flag = pd.merge(scenario1[["f.item"]], item_df[:item_df.shape[0]-1], left_on="f.item", right_on="item", how="left")[["id", "item"]]
element_flag = pd.merge(scenario1[["f.element"]], element_df[:element_df.shape[0]-1], 
                        left_on="f.element", right_on="element", how="left")[["id", "element"]]
company_flag = pd.merge(scenario1[["c.ticker"]], company_df, 
                        left_on="c.ticker", right_on="ticker", how="left")[["id", "ticker", "company"]]
index_get = np.array(np.where((pd.isna(item_flag["id"])==False) & 
                              (pd.isna(element_flag["id"])==False) & (pd.isna(company_flag["id"])==False))[0], dtype="int")
item_fc = np.array(item_flag["id"].iloc[index_get], dtype="int16")
element_fc = np.array(element_flag["id"].iloc[index_get], dtype="int16")
company_fc = np.array(company_flag["id"].iloc[index_get], dtype="int16")
N1 = index_get.shape[0]

#データフレームを作成
temp_item = item_df.iloc[item_fc]
temp_item.columns = ["item_id", "item"]; temp_item.index = np.arange(temp_item.shape[0])
temp_element = element_df.iloc[element_fc]
temp_element.columns = ["element_id", "element"]; temp_element.index = np.arange(temp_element.shape[0])
temp_company = company_df.iloc[company_fc]
temp_company.columns = ["company_id", "ticker", "company"]; temp_company.index = np.arange(temp_company.shape[0])
fc = pd.concat((temp_item, temp_element, temp_company), axis=1)
del temp_item, temp_element, temp_company

In [57]:
#要因-要因間のネットワークを生成
item_flag1 = pd.merge(factor[["f1.item"]], item_df[:item_df.shape[0]-1], left_on="f1.item", right_on="item", how="left")[["id", "item"]]
item_flag2 = pd.merge(factor[["f2.item"]], item_df[:item_df.shape[0]-1], left_on="f2.item", right_on="item", how="left")[["id", "item"]]
element_flag1 = pd.merge(factor[["f1.element"]], element_df[:element_df.shape[0]-1], 
                         left_on="f1.element", right_on="element", how="left")[["id", "element"]] 
element_flag2 = pd.merge(factor[["f2.element"]], element_df[:element_df.shape[0]-1], 
                         left_on="f2.element", right_on="element", how="left")[["id", "element"]]
index_get = np.array(np.where((pd.isna(item_flag1["id"])==False) & (pd.isna(item_flag2["id"])==False) &
                              (pd.isna(element_flag1["id"])==False) & (pd.isna(element_flag2["id"])==False))[0], dtype="int")
item_ff1 = np.array(item_flag1["id"].iloc[index_get], dtype="int16")
item_ff2 = np.array(item_flag2["id"].iloc[index_get], dtype="int16")
element_ff1 = np.array(element_flag1["id"].iloc[index_get], dtype="int16")
element_ff2 = np.array(element_flag2["id"].iloc[index_get], dtype="int16")
N2 = index_get.shape[0]

#データフレームを作成
temp_item1 = item_df.iloc[item_ff1]
temp_item1.columns = ["item_id1", "item1"]; temp_item1.index = np.arange(temp_item1.shape[0])
temp_item2 = item_df.iloc[item_ff2]
temp_item2.columns = ["item_id2", "item2"]; temp_item2.index = np.arange(temp_item2.shape[0])
temp_element1 = element_df.iloc[element_ff1]
temp_element1.columns = ["element_id1", "element1"]; temp_element1.index = np.arange(temp_element1.shape[0])
temp_element2 = element_df.iloc[element_ff2]
temp_element2.columns = ["element_id2", "element2"]; temp_element2.index = np.arange(temp_element2.shape[0])
ff = pd.concat((temp_item1, temp_element1, temp_item2, temp_element2), axis=1)
del temp_item1, temp_item2, temp_element1, temp_element2

In [58]:
#要因-要因-企業間のネットワークを生成
item_flag1 = pd.merge(scenario2[["f1.item"]], item_df[:item_df.shape[0]-1], left_on="f1.item", right_on="item", how="left")[["id", "item"]]
item_flag2 = pd.merge(scenario2[["f2.item"]], item_df[:item_df.shape[0]-1], left_on="f2.item", right_on="item", how="left")[["id", "item"]]
element_flag1 = pd.merge(scenario2[["f1.element"]], element_df[:element_df.shape[0]-1], 
                         left_on="f1.element", right_on="element", how="left")[["id", "element"]]
element_flag2 = pd.merge(scenario2[["f2.element"]], element_df[:element_df.shape[0]-1], 
                         left_on="f2.element", right_on="element", how="left")[["id", "element"]]
company_flag = pd.merge(scenario2[["c.ticker"]], company_df,
                        left_on="c.ticker", right_on="ticker", how="left")[["id", "ticker", "company"]]
index_get = np.array(np.where((pd.isna(item_flag1["id"])==False) & (pd.isna(item_flag2["id"])==False) &
                              (pd.isna(element_flag1["id"])==False) & (pd.isna(element_flag2["id"])==False) & 
                              (pd.isna(company_flag["id"])==False))[0], dtype="int")
item_ffc1 = np.array(item_flag1["id"].iloc[index_get], dtype="int16")
item_ffc2 = np.array(item_flag2["id"].iloc[index_get], dtype="int16")
element_ffc1 = np.array(element_flag1["id"].iloc[index_get], dtype="int16")
element_ffc2 = np.array(element_flag2["id"].iloc[index_get], dtype="int16")
company_ffc = np.array(company_flag["id"].iloc[index_get], dtype="int16")
N3 = index_get.shape[0]

#データフレームを作成
temp_item1 = item_df.iloc[item_ffc1]
temp_item1.columns = ["item_id1", "item1"]; temp_item1.index = np.arange(temp_item1.shape[0])
temp_item2 = item_df.iloc[item_ffc2]
temp_item2.columns = ["item_id2", "item2"]; temp_item2.index = np.arange(temp_item2.shape[0])
temp_element1 = element_df.iloc[element_ffc1]
temp_element1.columns = ["element_id1", "element1"]; temp_element1.index = np.arange(temp_element1.shape[0])
temp_element2 = element_df.iloc[element_ffc2]
temp_element2.columns = ["element_id2", "element2"]; temp_element2.index = np.arange(temp_element2.shape[0])
temp_company = company_df.iloc[company_ffc]
temp_company.columns = ["company_id", "ticker", "company"]; temp_company.index = np.arange(temp_company.shape[0])
ffc = pd.concat((temp_item1, temp_element1, temp_item2, temp_element2, temp_company), axis=1)
del temp_item1, temp_item2, temp_element1, temp_element2, temp_company

In [59]:
##要因-企業-要因のネットワークデータを生成
#要因と企業を抽出
item_flag = pd.merge(scenario1[["f.item"]], item_df, left_on="f.item", right_on="item", how="left")[["id", "item"]]
element_flag = pd.merge(scenario1[["f.element"]], element_df, left_on="f.element", right_on="element", how="left")[["id", "element"]]
company_flag = pd.merge(scenario1[["c.ticker"]], company_df, 
                        left_on="c.ticker", right_on="ticker", how="left")[["id", "ticker", "company"]]
index_get = np.array(np.where((pd.isna(item_flag["id"])==False) & 
                              (pd.isna(element_flag["id"])==False) & (pd.isna(company_flag["id"])==False))[0], dtype="int")

#データの格納用リスト
get_company = np.unique(company_fc)
n = get_company.shape[0]
company_list = [i for i in range(n)]
item_list1 = [i for i in range(n)]
item_list2 = [i for i in range(n)]
element_list1 = [i for i in range(n)]
element_list2 = [i for i in range(n)]

for i in range(n):
    #対象のitemとelementを抽出
    index = np.array(np.where(company_fc==get_company[i])[0], dtype="int")
    get_item = item_fc[index]
    get_element = element_fc[index]
    if get_element.shape[0] <= 1:
        company_list[i] = np.array([], dtype="int16")
        item_list1[i] = np.array([], dtype="int16")
        item_list2[i] = np.array([], dtype="int16")
        element_list1[i] = np.array([], dtype="int16")
        element_list2[i] = np.array([], dtype="int16")
        continue

    #組み合わせを作成
    m = get_item.shape[0]
    get_combine1 = np.repeat(np.arange(m-1), np.arange(m-1)[::-1]+1)
    index_del = np.array([], dtype="int")
    get_combine2 = np.array([], dtype="int")
    for j in range(m-1):
        index_del = np.append(index_del, j) 
        get_combine2 = np.append(get_combine2, np.delete(np.arange(m), index_del))

    #リストに格納
    company_list[i] = np.repeat(get_company[i], get_combine1.shape[0])
    item_list1[i] = get_item[get_combine1]
    item_list2[i] = get_item[get_combine2]
    element_list1[i] = get_element[get_combine1]
    element_list2[i] = get_element[get_combine2]
    
#リストを変換
company_fcf = np.array(list(itertools.chain(*[company_list[i] for i in range(n)])))
item_fcf1 = np.array(list(itertools.chain(*[item_list1[i] for i in range(n)])))
item_fcf2 = np.array(list(itertools.chain(*[item_list2[i] for i in range(n)])))
element_fcf1 = np.array(list(itertools.chain(*[element_list1[i] for i in range(n)])))
element_fcf2 = np.array(list(itertools.chain(*[element_list2[i] for i in range(n)])))
N4 = company_fcf.shape[0]

#データフレームを作成
temp_item1 = item_df.iloc[item_fcf1]
temp_item1.columns = ["item_id1", "item1"]; temp_item1.index = np.arange(temp_item1.shape[0])
temp_item2 = item_df.iloc[item_fcf2]
temp_item2.columns = ["item_id2", "item2"]; temp_item2.index = np.arange(temp_item2.shape[0])
temp_element1 = element_df.iloc[element_fcf1]
temp_element1.columns = ["element_id1", "element1"]; temp_element1.index = np.arange(temp_element1.shape[0])
temp_element2 = element_df.iloc[element_fcf2]
temp_element2.columns = ["element_id2", "element2"]; temp_element2.index = np.arange(temp_element2.shape[0])
temp_company = company_df.iloc[company_fcf]
temp_company.columns = ["company_id", "ticker", "company"]; temp_company.index = np.arange(temp_company.shape[0])
fcf = pd.concat((temp_company, temp_item1, temp_element1, temp_item2, temp_element2), axis=1)
del item_list1, item_list2, element_list1, element_list2, company_list
del temp_item1, temp_item2, temp_element1, temp_element2, temp_company

In [60]:
##企業-要因-企業のネットワークデータを生成
#要因と企業を抽出
item_flag = pd.merge(scenario1[["f.item"]], item_df, left_on="f.item", right_on="item", how="left")[["id", "item"]]
element_flag = pd.merge(scenario1[["f.element"]], element_df, left_on="f.element", right_on="element", how="left")[["id", "element"]]
company_flag = pd.merge(scenario1[["c.ticker"]], company_df, 
                        left_on="c.ticker", right_on="ticker", how="left")[["id", "ticker", "company"]]
index_get = np.array(np.where((pd.isna(item_flag["id"])==False) & 
                              (pd.isna(element_flag["id"])==False) & (pd.isna(company_flag["id"])==False))[0], dtype="int")

#データの格納用リスト
get_item = np.unique(item_fc); get_item = get_item[:get_item.shape[0]-1]
n = get_item.shape[0]
item_list = [i for i in range(n)]
company_list1 = [i for i in range(n)]
company_list2 = [i for i in range(n)]

for i in range(n):
    #対象の企業を抽出
    index = np.array(np.where(item_fc==get_item[i])[0], dtype="int")
    get_company = company_fc[index]
    if get_element.shape[0] <= 1:
        item_list[i] = np.array([], dtype="int16")
        company_list1[i] = np.array([], dtype="int16")
        company_list2[i] = np.array([], dtype="int16")
        continue

    #組み合わせを作成
    m = get_company.shape[0]
    get_combine1 = np.repeat(np.arange(m-1), np.arange(m-1)[::-1]+1)
    index_del = np.array([], dtype="int")
    get_combine2 = np.array([], dtype="int")
    for j in range(m-1):
        index_del = np.append(index_del, j) 
        get_combine2 = np.append(get_combine2, np.delete(np.arange(m), index_del))

    #リストに格納
    item_list[i] = np.repeat(get_item[i], get_combine1.shape[0])
    company_list1[i] = get_company[get_combine1]
    company_list2[i] = get_company[get_combine2]
    
#リストを変換
item_cfc = np.array(list(itertools.chain(*[item_list[i] for i in range(n)])))
company_cfc1 = np.array(list(itertools.chain(*[company_list1[i] for i in range(n)])))
company_cfc2 = np.array(list(itertools.chain(*[company_list2[i] for i in range(n)])))
N5 = item_cfc.shape[0]

#データフレームを作成
temp_item = item_df.iloc[item_cfc]
temp_item.columns = ["item_id", "item"]; temp_item.index = np.arange(temp_item.shape[0])
temp_company1 = company_df.iloc[company_cfc1]
temp_company1.columns = ["company_id1", "ticker1", "company1"]; temp_company1.index = np.arange(temp_company1.shape[0])
temp_company2 = company_df.iloc[company_cfc2]
temp_company2.columns = ["company_id2", "ticker2", "company2"]; temp_company2.index = np.arange(temp_company2.shape[0])
cfc = pd.concat((temp_company1, temp_company2, temp_item), axis=1)
del item_list, company_list1, company_list2
del temp_item, temp_company1, temp_company2

In [61]:
#データフレームを出力
fc.to_excel("D:/Statistics/data/scenario_link/factor-company.xlsx")
ff.to_excel("D:/Statistics/data/scenario_link/factor-factor.xlsx")
ffc.to_excel("D:/Statistics/data/scenario_link/factor-factor-company.xlsx")
fcf.to_excel("D:/Statistics/data/scenario_link/factor-company-factor.xlsx")
cfc.to_excel("D:/Statistics/data/scenario_link/company-factor-company.xlsx")

In [77]:
##負例サンプリングにより新しいサンプルを生成
#サンプリング確率を設定
item_prob = np.power(item_freq["freq"], 0.75) / np.sum(np.power(item_freq["freq"], 0.75))
element_prob = np.power(element_freq["freq"], 0.75) / np.sum(np.power(element_freq["freq"], 0.75))
company_prob = np.power(company_freq["freq"], 0.75) / np.sum(np.power(company_freq["freq"], 0.75))

#要因-企業間の負例をサンプリング

N1

7791

0       0.023956
1       0.019488
2       0.009815
3       0.008814
4       0.007169
5       0.007106
6       0.005928
7       0.005702
8       0.004942
9       0.004866
10      0.004617
11      0.004399
12      0.004191
13      0.004137
14      0.004042
15      0.003936
16      0.003902
17      0.003877
18      0.003813
19      0.003694
20      0.003694
21      0.003373
22      0.003248
23      0.003121
24      0.003029
25      0.002875
26      0.002871
27      0.002851
28      0.002664
29      0.002655
          ...   
3529    0.000060
3530    0.000060
3531    0.000060
3532    0.000060
3533    0.000060
3534    0.000060
3535    0.000060
3536    0.000060
3537    0.000060
3538    0.000060
3539    0.000060
3540    0.000060
3541    0.000060
3542    0.000060
3543    0.000060
3544    0.000060
3545    0.000060
3546    0.000060
3547    0.000060
3548    0.000060
3549    0.000060
3550    0.000060
3551    0.000060
3552    0.000060
3553    0.000060
3554    0.000060
3555    0.000060
3556    0.0000